In [ ]:
import numpy as np
import pandas as pd

import os
import sys
sys.path.insert(0, '..')
import ccal

%matplotlib inline
# %config InlineBackend.figure_formats = {'svg',}

# Get states

In [ ]:
h = ccal.support.read_gct('/home/cyborg/hematopoietic_cancer/result/ccle_hema_r/k9/ccle_hema_k_9_H.gct')
n_states_to_explore = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 25, 30]
nclustering = 500
filename_prefix = 'output/ccle_hema_k_9_H_1000clustering_0828'

In [ ]:
# labels, memberships, cophenetic_correlation = ccal.analyze.get_states_from_h(h, n_states_to_explore, nclustering=nclustering, filename_prefix=filename_prefix)

# Map Onco-GPS

In [ ]:
labels = pd.read_csv(filename_prefix + '_labels.txt', index_col=0, sep='\t')
memberships = ccal.support.read_gct(filename_prefix + '_memberships.gct')
annotations = ccal.support.make_random_features(1, memberships.shape[1])

In [ ]:
ccal.visualize.plot_onco_gps(h,
                             np.random.choice(range(10), h.shape[1]))#,
#                              background_markersize=0, background_mask_markersize=0)
ccal.visualize.plot_onco_gps(h,
                             np.random.choice(range(10), h.shape[1]))#,
#                              background_markersize=0, background_mask_markersize=0, mds_is_metric=False)
ccal.visualize.plot_onco_gps(h,
                             np.random.choice(range(10), h.shape[1]))#,
#                              background_markersize=0, background_mask_markersize=0, informational_mds=False)
ccal.visualize.plot_onco_gps(h,
                             np.random.choice(range(10), h.shape[1]),
                             annotations=np.random.random_sample(h.shape[1]))#
#                              background_markersize=0, background_mask_markersize=0)
ccal.visualize.plot_onco_gps(h,
                             np.random.choice(range(10), h.shape[1]),
                             annotations=np.random.choice(range(5), h.shape[1]),
                             annotation_type='categorical')#
#                              background_markersize=0, background_mask_markersize=0)
ccal.visualize.plot_onco_gps(h,
                             np.random.choice(range(10), h.shape[1]),
                             annotations=np.random.choice(range(2), h.shape[1]),
                             annotation_type='binary')#
#                              background_markersize=0, background_mask_markersize=0)

In [ ]:
for n_state in [12]:# n_states_to_explore:
    states = labels.ix[n_state, :-1].astype(int).tolist()
    signature = 'mds_informational_metric'
    ccal.visualize.plot_onco_gps(h, states, mds_is_metric=True, informational_mds=True, output_filepath='output/ccle_hema_k_9_H_1000clustering_0828_s{}_{}.pdf'.format(n_state, signature))
    signature = 'mds_informational_no_metric'
    ccal.visualize.plot_onco_gps(h, states, mds_is_metric=False, informational_mds=True, output_filepath='output/ccle_hema_k_9_H_1000clustering_0828_s{}_{}.pdf'.format(n_state, signature))
    signature = 'mds_no_informational_metric'
    ccal.visualize.plot_onco_gps(h, states, mds_is_metric=True, informational_mds=False, output_filepath='output/ccle_hema_k_9_H_1000clustering_0828_s{}_{}.pdf'.format(n_state, signature))
    signature = 'mds_no_informational_no_metric'
    ccal.visualize.plot_onco_gps(h, states, mds_is_metric=False, informational_mds=False, output_filepath='output/ccle_hema_k_9_H_1000clustering_0828_s{}_{}.pdf'.format(n_state, signature))
#     signature = 'populate_random'
#     ccal.visualize.plot_onco_gps(h, states, annotations=annotations, output_filepath='output/ccle_hema_k_9_H_1000clustering_0828_s{}_{}.pdf'.format(n_state, signature))

In [2]:
print_log = print

In [17]:
def establish_path(filepath):
    """
    Make directories up to `fullpath` if they don't already exist.
    :param filepath: str;
    :return: None
    """
    from os import path, mkdir
    
    prefix, suffix = path.split(filepath)
    if not (path.isdir(prefix) or path.isfile(prefix) or path.islink(prefix)):
        print_log('Directory {} doesn\'t exist, creating it ...'.format(prefix))
        dirs = []
        prefix, suffix = path.split(filepath)
        dirs.append(prefix)
        while prefix != '' and suffix != '':
            prefix, suffix = path.split(prefix)
            if prefix:
                dirs.append(prefix)
        for d in reversed(dirs):
            if not (path.isdir(d) or path.isfile(d) or path.islink(d)):
                mkdir(d)
                print_log('Created directory {}.'.format(d))

establish_path('Desktopfff/')

Directory Desktopfff doesn't exist, creating it ...
Created directory Desktopfff.
